In [3]:
from __future__ import absolute_import
from __future__ import print_function
from future.standard_library import install_aliases
install_aliases()

import os
import gzip
import struct
import array
import numpy as np
from urllib.request import urlretrieve

def download(url, filename):
    if not os.path.exists('data'):
        os.makedirs('data')
    out_file = os.path.join('data', filename)
    if not os.path.isfile(out_file):
        urlretrieve(url, out_file)

def mnist():
    base_url = 'http://yann.lecun.com/exdb/mnist/'

    def parse_labels(filename):
        with gzip.open(filename, 'rb') as fh:
            magic, num_data = struct.unpack(">II", fh.read(8))
            return np.array(array.array("B", fh.read()), dtype=np.uint8)

    def parse_images(filename):
        with gzip.open(filename, 'rb') as fh:
            magic, num_data, rows, cols = struct.unpack(">IIII", fh.read(16))
            return np.array(array.array("B", fh.read()), dtype=np.uint8).reshape(num_data, rows, cols)

    for filename in ['train-images-idx3-ubyte.gz',
                     'train-labels-idx1-ubyte.gz',
                     't10k-images-idx3-ubyte.gz',
                     't10k-labels-idx1-ubyte.gz']:
        download(base_url + filename, filename)

    train_images = parse_images('data/train-images-idx3-ubyte.gz')
    train_labels = parse_labels('data/train-labels-idx1-ubyte.gz')
    test_images  = parse_images('data/t10k-images-idx3-ubyte.gz')
    test_labels  = parse_labels('data/t10k-labels-idx1-ubyte.gz')

    return train_images, train_labels, test_images, test_labels

def load_mnist():
    partial_flatten = lambda x : np.reshape(x, (x.shape[0], np.prod(x.shape[1:])))
    one_hot = lambda x, k: np.array(x[:,None] == np.arange(k)[None, :], dtype=int)
    train_images, train_labels, test_images, test_labels = mnist()
    train_images = partial_flatten(train_images) / 255.0
    test_images  = partial_flatten(test_images)  / 255.0
    train_labels = one_hot(train_labels, 10)
    test_labels = one_hot(test_labels, 10)

    return train_images, train_labels, test_images, test_labels

def check_my_network(layer_sizes, network):
    correct = True
    if not len(network)+1 == len(layer_sizes):
        correct = False
        print("The network does not have the correct number of layers")
        print("Number of layers in network: ", len(network))
        print("Expected number of layers: ", len(layer_sizes))
    for i in range(len(network)):
        if not network[i][0].shape == (layer_sizes[i], layer_sizes[i+1]):
            correct = False
            print("Layer " + str(i) + " does not have the correct dimension weights")
            print("Layer weight shape: ", str(network[i][0].shape))
            print("Expected shape: ", str((layer_sizes[i], layer_sizes[i+1])))
            print("################################################################")
        if not network[i][1].shape[0] == layer_sizes[i+1]:
            correct = False
            print("Layer " + str(i) + " does not have the correct dimension biases")
            print("Layer bias shape: ", str(network[i][1].shape[0]))
            print("Expected shape: ", str(layer_sizes[i+1]))
            print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
    if correct:
        print("The network is correct")


# Above is from the helpers.py

In [4]:
import random
import math

### Layer of a network:

In [5]:
class init_layer:
    
    def __init__(self, x , y):
        #x is the number of neurons in this layer
        #y is the number of neurons in the next layer
        self.weight_matrix = np.random.rand(x,y) * 0.01
        self.bias_vector = np.random.rand(y)
    
        #we need a way to store a neuron
        
        

In [6]:
class init_neuron:
    
    def __init__(self, x):
        self.neurons_curr_layer = np.zeros(x) #the neurons in this layer, all set to zero
    
    def len(self):
        return len(self.neurons_curr_layer)
    
    def set_inputs(self, input_data):
        self.neurons_curr_layer = input_data
        


### Me checking that the layer works

In [7]:
newLayer = init_layer(10,5)
print("The weight matrix")
print(newLayer.weight_matrix)
print("The bias vector")
print(newLayer.bias_vector)

The weight matrix
[[0.00348985 0.00812805 0.0011995  0.00053321 0.00289731]
 [0.00256927 0.00507628 0.00947283 0.00070372 0.00016483]
 [0.00969072 0.00415079 0.00172551 0.00738362 0.00694435]
 [0.00932318 0.00441869 0.00473864 0.00210087 0.00869308]
 [0.00920258 0.00928364 0.00832832 0.00200992 0.00559429]
 [0.00150893 0.00440199 0.00169941 0.00484246 0.00390843]
 [0.00506402 0.00154886 0.00217287 0.00294025 0.00917628]
 [0.00091926 0.00437389 0.008851   0.00847716 0.00743286]
 [0.00852752 0.00344488 0.0045205  0.00886953 0.000953  ]
 [0.00795812 0.00921771 0.00226704 0.00906772 0.00672025]]
The bias vector
[0.28346807 0.0404262  0.83513583 0.31723678 0.41011025]


### The network itself

In [8]:
class init_random_network:
    
    def __init__(self, network_info):
        self.layers = []
        for i in range(len(network_info) -1):
            self.layers.append(init_layer(network_info[i],network_info[i+1]))
        
        self.neurons = []
        for i in range(len(network_info)):
                self.neurons.append(init_neuron(network_info[i]))
    
    def assignData(self, data, index):
        data_point = data[index]
        print(data_point.shape)
        self.neurons[0].set_inputs(data_point)
        neurons = self.neurons[0]
        
                
    def output_at_layer(self, position):
        try:
            return self.layers[position].weight_matrix, self.layers[position].bias_vector
        except:
            if position > len(layers):
                print("out of index")
            else:
                print("some error occured")

The network being created for an example

In [9]:
random_network = init_random_network([784,200,100,10])

Running through the network to show its properties match the desired properties

In [10]:
for i in range(len(random_network.layers)):
    print("this is the network layer's weight matrix " + str(random_network.layers[i].weight_matrix.shape))
    print("This is the network layer's bias vector " + str(random_network.layers[i].bias_vector.shape))
    print("number of neurons in this layer "+ str(random_network.neurons[i].len()))

this is the network layer's weight matrix (784, 200)
This is the network layer's bias vector (200,)
number of neurons in this layer 784
this is the network layer's weight matrix (200, 100)
This is the network layer's bias vector (100,)
number of neurons in this layer 200
this is the network layer's weight matrix (100, 10)
This is the network layer's bias vector (10,)
number of neurons in this layer 100


In [11]:
data = load_mnist()
#Data takes the form of a tuple
#where the touple is the  (data,labels)
trainingData = np.array([data[0][0],data[0][1]])
trainingLabels = np.array([data[1][0],data[1][1]])
training = [trainingData, trainingLabels]
testData = np.array([data[2][0],data[2][1]])
testLabel = np.array([data[3][0],data[3][1]])

# TODO: part d, make forward propagation function

In [12]:
# function that propagates a batch of data through the neural network
# accepts list of tuples : these specify the network parameters (like random_networks)
# also accepts the data
# GOAL: produce output from network for each data point
# activation must be a sigmoid function on all layers excluding the last
# array shape at the end is (60 000, 10)
# brightest final neuron is the number we are (our answer)

In [13]:
# to push our inputs to a [0,1]
# do not use on the last layer...
def sigmoid(x):
    return 1/(1+ math.exp(-x))

In [14]:
def array_for(x):
    return np.array([sigmoid(xi) for xi in x])

In [17]:
def getInput(data, index):
    return data[index]

In [57]:
def propagate_data(network_info, data_input):
    network = init_random_network(network_info)
    size = data_input.shape[0]
    output = np.zeros(shape = (60000, 10))
    #print(size)
    for i in range(size): #meant to serve as way to itterate over the 60 000 data points
                  
        data_point = getInput(data_input, i) #the data_point, so our input
        #print("i is " + str(i))
        for j in range(len(network_info) - 1): # final point is just output (have 10 options), therefore -1 from length
            #next input is current input dot producted with weight matrix, activated then added with the bias
            #print("j is " + str(j))
           #print(network.layers[j].weight_matrix.shape)
            data_point = array_for(np.dot(data_point, network.layers[j].weight_matrix) + network.layers[j].bias_vector)
            
        output[i,:] = data_point
        
    return output
        
    
    #z = np.dot(weight_matrix[i], x)
    #a = sigmoid(z)

In [60]:
data[1][1]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [59]:
output = propagate_data([784,200,100,10],data[0])

In [72]:
output[1]

array([0.78877456, 0.76974103, 0.61259289, 0.75811699, 0.77399541,
       0.60921616, 0.75661306, 0.63727602, 0.75871059, 0.73693705])

In [69]:
def accuracy(trainingAnswers, realAnswers):
    
    trainAnsIndex = np.argmax(trainingAnswers, axis = 1)
    realAnsIndex = np.argmax(realAnswers, axis = 1)
    
    numberCorrect = 0
    numberWrong = 0
    for i in range(len(trainAnsIndex)):
        if trainAnsIndex[i] == realAnsIndex[i]:
            numberCorrect = numberCorrect + 1
        else:
            numberWrong = numberWrong + 1
    
    print("the number of correct guesses is " + str(numberCorrect))
    print("the number of incorrect guesses is " + str(numberWrong))
    print(str(numberCorrect/len(trainAnsIndex) * 100) + "% correct")
        

In [70]:
accuracy(output, data[1])

the number of correct guesses is 5923
the number of incorrect guesses is 54077
9.871666666666666% correct
